In [8]:
import sqlalchemy as sa
import psycopg2
import pandas as pd
import shapely.wkt
import geopandas as gpd

In [9]:
eng = sa.create_engine("postgresql://osmuser@localhost:5432/osm")

In [10]:
import ipywidgets as widgets

In [11]:
import pydeck as pdk
import geopandas as gpdx

def f(input_text):
    try:
        query, geographic_reference = input_text.split('in')
    except Exception:
        raise InputError("Current query requires an `in` e.g. `book stores in san francisco`")
    
    SQL = f"""
    WITH container AS (
        SELECT geom
        FROM (
        SELECT ST_BuildArea(geom) AS geom
        FROM boundaries
        WHERE fts @@ websearch_to_tsquery('{geographic_reference}')
        LIMIT 5
        ) candidates
        ORDER BY ST_Area(geom) DESC
        LIMIT 1
    )
    SELECT
        ST_AsText(ST_Transform(points.geom, 4326)) as geometry
    , json_build_object(
        'name', tags->>'name',
        'amenity', tags->>'amenity') as properties
    FROM points
    JOIN container
    ON ST_CONTAINS(container.geom, points.geom)
    WHERE 1=1
        AND fts @@ websearch_to_tsquery('{query}')
    """
    
    df = pd.read_sql(SQL, eng)
    gdf = gpd.GeoDataFrame(df)
    
    
    gdf['geometry'] = gdf['geometry'].apply(lambda x: shapely.wkt.loads(x))
    gdf['label'] = gdf['properties'].apply(lambda x: x.get('name'))
    
    INITIAL_VIEW_STATE = {
        "longitude": -122.419906,
        "latitude": 37.7790262,
        "zoom": 10.656177979169486,
        "pitch": 0,
        "bearing": 0
    }
    
    layers = [
        pdk.Layer(
            "GeoJsonLayer",
            data=gdf,
            get_radius=200,
            get_fill_color=[0, 150, 255],
            filled=True,
            line_width_min_pixels=5,
            radius_min_pixels=5,
            opacity=0.4,
            stroked=False,
            extruded=True,
            wireframe=True,
            pickable=True,
        )       
    ]
    
    d = pdk.Deck(layers,
        initial_view_state=INITIAL_VIEW_STATE,
        tooltip={
            "html": "{label}"
        }
    )
    return d

In [12]:
from IPython.display import display, HTML, IFrame, clear_output
from ipywidgets import widgets, Layout, HBox

html_out = widgets.Output(layout=Layout(width="100%"))


h_box = HBox([html_out])

output = widgets.Output()


t = widgets.Text(
    value='library in san francisco',
    placeholder='Type something',
    description='Search:',
    disabled=False
)
        
def on_button_clicked(b):
    with html_out:
        clear_output(wait=True)
        display(f(t.value))
        
button = widgets.Button(description="Click Me!")
button.on_click(on_button_clicked)

display(t)
display(button)
display(h_box)

Text(value='library in san francisco', description='Search:', placeholder='Type something')

Button(description='Click Me!', style=ButtonStyle())